In [1]:
import re
import os
from datetime import date
from bs4 import BeautifulSoup
from selenium import webdriver
from colored import fg, bg, attr
from collections import defaultdict as dd
import numpy as np
import pandas as pd

In [2]:
news_list = []
player_news_list = []
failures = [0]

In [3]:
nowx = pd.datetime.now()
dates = [x for x in pd.date_range(start ='2016-09-01', end='2017-02-05')] +
        [x for x in pd.date_range(start ='2017-09-01', end='2017-11-17')]
index = len(dates) - 1
driver = webdriver.Firefox()
driver.get('http://www.rotoworld.com/playernews/nfl/football-player-news')
while index >= 0:
    x = dates[index]
    nowx = pd.datetime.now()
    print('Scraping:', str(x.month) + '/' + str(x.day) + '/' + str(x.year))
    team_news = dd(lambda : dd(list))
    try:
        date_picker = driver.find_element_by_name('ctl00$cp1$ctl00$tbDatepicker')
        date_picker.click()
        date_picker.clear()
        date_picker.send_keys('{}/{}/{}'.format(x.month, x.day, x.year))
        driver.find_element_by_name('ctl00$cp1$ctl00$btnFilterResults').click()
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        player_boxes = soup.select('.pb')
        for player_box in player_boxes:
            date_str = player_box.select('.date')[0].contents[0]
            date = int(re.findall(r'^.* (\d+)[ ,].*$', date_str)[0])
            player_dict = {}
            player_info = player_box.select('.player a')
            player_dict['team'] = player_info[1].contents[0]
            player_dict['name'] = player_info[0].contents[0]
            player_dict['news'] = (player_box.select('p')[0].contents[0],
                                  player_box.select('.impact')[0].contents[0])
            player_dict['date'] = x
            player_news_list.append(player_dict)
        print('success')
    except:
        failures.append(x)
        print('failed')
    index -= 1
driver.close()
test = pd.DataFrame(player_news_list)
test.to_excel('all_season.xlsx')

Scraping: 11/17/2017
success
Scraping: 11/16/2017
success
Scraping: 11/15/2017
success
Scraping: 11/14/2017
success
Scraping: 11/13/2017
success
Scraping: 11/12/2017
success
Scraping: 11/11/2017
success
Scraping: 11/10/2017
success
Scraping: 11/9/2017
success
Scraping: 11/8/2017
success
Scraping: 11/7/2017
success
Scraping: 11/6/2017
success
Scraping: 11/5/2017
success
Scraping: 11/4/2017
success
Scraping: 11/3/2017
success
Scraping: 11/2/2017
success
Scraping: 11/1/2017
success
Scraping: 10/31/2017
success
Scraping: 10/30/2017
success
Scraping: 10/29/2017
success
Scraping: 10/28/2017
success
Scraping: 10/27/2017
success
Scraping: 10/26/2017
success
Scraping: 10/25/2017
success
Scraping: 10/24/2017
success
Scraping: 10/23/2017
success
Scraping: 10/22/2017
success
Scraping: 10/21/2017
success
Scraping: 10/20/2017
success
Scraping: 10/19/2017
success
Scraping: 10/18/2017
success
Scraping: 10/17/2017
success
Scraping: 10/16/2017
success
Scraping: 10/15/2017
success
Scraping: 10/14/2017
su

success
Scraping: 1/26/2017
success
Scraping: 1/25/2017
success
Scraping: 1/24/2017
success
Scraping: 1/23/2017
success
Scraping: 1/22/2017
success
Scraping: 1/21/2017
success
Scraping: 1/20/2017
success
Scraping: 1/19/2017
success
Scraping: 1/18/2017
success
Scraping: 1/17/2017
success
Scraping: 1/16/2017
success
Scraping: 1/15/2017
success
Scraping: 1/14/2017
success
Scraping: 1/13/2017
success
Scraping: 1/12/2017
success
Scraping: 1/11/2017
success
Scraping: 1/10/2017
success
Scraping: 1/9/2017
success
Scraping: 1/8/2017
success
Scraping: 1/7/2017
success
Scraping: 1/6/2017
success
Scraping: 1/5/2017
success
Scraping: 1/4/2017
success
Scraping: 1/3/2017
success
Scraping: 1/2/2017
success
Scraping: 1/1/2017
success
Scraping: 12/31/2016
success
Scraping: 12/30/2016
success
Scraping: 12/29/2016
success
Scraping: 12/28/2016
success
Scraping: 12/27/2016
success
Scraping: 12/26/2016
success
Scraping: 12/25/2016
success
Scraping: 12/24/2016
success
Scraping: 12/23/2016
success
Scraping: 12

WebDriverException: Message: Tried to run command without establishing a connection


In [4]:
len(player_news_list)

5665

In [5]:
len(failures)

55

In [6]:
len(dates)

443

In [7]:
test = pd.DataFrame(player_news_list)
test.to_excel('all_season.xlsx')

In [ ]:
class RWScraper(object):
    '''

    '''
    def __init__(self):
        self.url = 'http://www.rotoworld.com/playernews/nfl/football-player-news'
        self._team_news = dd(lambda : dd(list))
        self._same_day = True
        self._date = (0, date.today().day, 0)

    def _get_player_info(self, player_box):
        date_str = player_box.select('.date')[0].contents[0]
        date = int(re.findall(r'^.* (\d+)[ ,].*$', date_str)[0])
        if date != self._date[1]:
            return False
        player_dict = {}
        player_info = player_box.select('.player a')
        player_dict['team'] = player_info[1].contents[0]
        player_dict['name'] = player_info[0].contents[0]
        player_dict['news'] = (player_box.select('p')[0].contents[0], 
                               player_box.select('.impact')[0].contents[0])
        return player_dict

    def _boxes_to_news(self, player_boxes):
        for player_box in player_boxes:
            player_dict = self._get_player_info(player_box)
            if not player_dict:
                return False
            self._team_news[player_dict['team']][player_dict['name']].append(player_dict['news'])
        return True

    def _get_player_news(self):
        driver = webdriver.PhantomJS()
        driver.get(self.url)
        if self._date[0]:
            date_picker = driver.find_element_by_id('tbDatepicker')
            date_picker.click()
            date_picker.send_keys('{}/{}/{}'.format(self._date[0], self._date[1], self._date[2]))
            driver.find_element_by_id('cp1_ctl00_btnFilterResults').click()
        while self._same_day:
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            player_boxes = soup.select('.pb')
            self._same_day = self._boxes_to_news(player_boxes)
            driver.find_element_by_id('cp1_ctl00_btnNavigate1').click()

    def scrape(self, date='today'):
        if date != 'today':
            self._date = date
        self._get_player_news()
        os.remove('ghostdriver.log')